# Wrangle county-level census data

In [24]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
from shapely.geometry import Polygon
from shapely.geometry import mapping
import numpy as np


import warnings
warnings.simplefilter(action='ignore')

### Health Insurance Coverage Status By Sex By Age (2018 5-year estimates)

In [25]:
# read insurance coverage data in
insurance = pd.read_csv('../data/census-tables/2018-county-insurance-data.csv', encoding='latin-1', header=1)
list(insurance.columns)

['id',
 'Geographic Area Name',
 'Estimate!!Total',
 'Margin of Error!!Total',
 'Estimate!!Total!!Male',
 'Margin of Error!!Total!!Male',
 'Estimate!!Total!!Male!!Under 6 years',
 'Margin of Error!!Total!!Male!!Under 6 years',
 'Estimate!!Total!!Male!!Under 6 years!!With health insurance coverage',
 'Margin of Error!!Total!!Male!!Under 6 years!!With health insurance coverage',
 'Estimate!!Total!!Male!!Under 6 years!!No health insurance coverage',
 'Margin of Error!!Total!!Male!!Under 6 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!6 to 18 years',
 'Margin of Error!!Total!!Male!!6 to 18 years',
 'Estimate!!Total!!Male!!6 to 18 years!!With health insurance coverage',
 'Margin of Error!!Total!!Male!!6 to 18 years!!With health insurance coverage',
 'Estimate!!Total!!Male!!6 to 18 years!!No health insurance coverage',
 'Margin of Error!!Total!!Male!!6 to 18 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!19 to 25 years',
 'Margin of Error!!Total!!Male!!19 to 

I only want to keep the identifying columns, and I ultimately need a column for total uninsured.

In [26]:
# first filter to keep columns that contain estimate, to eliminate the margin of error columns
colNames = insurance.columns.str.contains('id|Geographic Area Name|Estimate', case=False)
insurance_filter = insurance.iloc[:, colNames]
list(insurance_filter.columns)

['id',
 'Geographic Area Name',
 'Estimate!!Total',
 'Estimate!!Total!!Male',
 'Estimate!!Total!!Male!!Under 6 years',
 'Estimate!!Total!!Male!!Under 6 years!!With health insurance coverage',
 'Estimate!!Total!!Male!!Under 6 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!6 to 18 years',
 'Estimate!!Total!!Male!!6 to 18 years!!With health insurance coverage',
 'Estimate!!Total!!Male!!6 to 18 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!19 to 25 years',
 'Estimate!!Total!!Male!!19 to 25 years!!With health insurance coverage',
 'Estimate!!Total!!Male!!19 to 25 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!26 to 34 years',
 'Estimate!!Total!!Male!!26 to 34 years!!With health insurance coverage',
 'Estimate!!Total!!Male!!26 to 34 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!35 to 44 years',
 'Estimate!!Total!!Male!!35 to 44 years!!With health insurance coverage',
 'Estimate!!Total!!Male!!35 to 44 years!!No health insurance

In [27]:
# create list of all columns that describe a group with no health coverage
colNames = insurance_filter.columns.str.contains('No', case=False)
no_insurance = insurance_filter.iloc[:, colNames]
col_list = list(no_insurance.columns)
col_list

['Estimate!!Total!!Male!!Under 6 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!6 to 18 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!19 to 25 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!26 to 34 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!35 to 44 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!45 to 54 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!55 to 64 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!65 to 74 years!!No health insurance coverage',
 'Estimate!!Total!!Male!!75 years and over!!No health insurance coverage',
 'Estimate!!Total!!Female!!Under 6 years!!No health insurance coverage',
 'Estimate!!Total!!Female!!6 to 18 years!!No health insurance coverage',
 'Estimate!!Total!!Female!!19 to 25 years!!No health insurance coverage',
 'Estimate!!Total!!Female!!26 to 34 years!!No health insurance coverage',
 'Estimate!!Total!!Female!!35 to 44 years!!No health insu

In [28]:
# create new field that is the sum of all columns describing groups with no health coverage
insurance_filter['totalUninsured'] = insurance_filter[col_list].sum(axis=1)
insurance_filter.head()

,id,Geographic Area Name,Estimate!!Total,Estimate!!Total!!Male,Estimate!!Total!!Male!!Under 6 years,Estimate!!Total!!Male!!Under 6 years!!With health insurance coverage,Estimate!!Total!!Male!!Under 6 years!!No health insurance coverage,Estimate!!Total!!Male!!6 to 18 years,Estimate!!Total!!Male!!6 to 18 years!!With health insurance coverage,Estimate!!Total!!Male!!6 to 18 years!!No health insurance coverage,...,Estimate!!Total!!Female!!55 to 64 years,Estimate!!Total!!Female!!55 to 64 years!!With health insurance coverage,Estimate!!Total!!Female!!55 to 64 years!!No health insurance coverage,Estimate!!Total!!Female!!65 to 74 years,Estimate!!Total!!Female!!65 to 74 years!!With health insurance coverage,Estimate!!Total!!Female!!65 to 74 years!!No health insurance coverage,Estimate!!Total!!Female!!75 years and over,Estimate!!Total!!Female!!75 years and over!!With health insurance coverage,Estimate!!Total!!Female!!75 years and over!!No health insurance coverage,totalUninsured
0,0500000US28151,"Washington County, Mississippi",46637,21772,2006,1893,113,4563,4280,283,...,3544,2906,638,2326,2326,0,1612,1612,0,8506
1,0500000US28111,"Perry County, Mississippi",11920,5795,409,395,14,1095,1014,81,...,852,676,176,677,677,0,434,434,0,1801
2,0500000US28019,"Choctaw County, Mississippi",8195,4075,336,336,0,838,786,52,...,598,530,68,504,504,0,467,467,0,694
3,0500000US28057,"Itawamba County, Mississippi",23252,11723,758,717,41,2231,2135,96,...,1438,1308,130,1189,1189,0,950,950,0,2642
4,0500000US28015,"Carroll County, Mississippi",9963,4989,274,274,0,1005,1005,0,...,788,742,46,728,728,0,426,426,0,841


In [29]:
# create new insurance dataframe by filtering only the columns I need
insurance_coverage = insurance_filter.filter(['id', 'Geographic Area Name', 'totalUninsured'])

# rename columns
insurance_coverage = insurance_coverage.rename(columns={'Geographic Area Name': 'name'})
insurance_coverage.head()

,id,name,totalUninsured
0,0500000US28151,"Washington County, Mississippi",8506
1,0500000US28111,"Perry County, Mississippi",1801
2,0500000US28019,"Choctaw County, Mississippi",694
3,0500000US28057,"Itawamba County, Mississippi",2642
4,0500000US28015,"Carroll County, Mississippi",841


In [30]:
# view insurance coverage statistics
insurance_coverage.describe()

,totalUninsured
count,3.220000e+03
mean,9.305143e+03
std,3.738955e+04
min,2.000000e+00
25%,9.747500e+02
50%,2.366000e+03
75%,5.966000e+03
max,1.086657e+06


### Poverty Status in the past 12 months (2018 5-year estimates)

In [31]:
# read poverty data in
poverty = pd.read_csv('../data/census-tables/2018-county-poverty-data.csv', encoding='latin-1', header=1)
list(poverty.columns)

['id',
 'Geographic Area Name',
 'Estimate!!Total!!Population for whom poverty status is determined',
 'Margin of Error!!Total MOE!!Population for whom poverty status is determined',
 'Estimate!!Below poverty level!!Population for whom poverty status is determined',
 'Margin of Error!!Below poverty level MOE!!Population for whom poverty status is determined',
 'Estimate!!Percent below poverty level!!Population for whom poverty status is determined',
 'Margin of Error!!Percent below poverty level MOE!!Population for whom poverty status is determined',
 'Estimate!!Total!!Population for whom poverty status is determined!!AGE!!Under 18 years',
 'Margin of Error!!Total MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years',
 'Estimate!!Below poverty level!!Population for whom poverty status is determined!!AGE!!Under 18 years',
 'Margin of Error!!Below poverty level MOE!!Population for whom poverty status is determined!!AGE!!Under 18 years',
 'Estimate!!Percent below pov

I just want to keep the identifying fields, and there is already a column for the percent below poverty level so I will not need to calculate that field from population totals.

In [32]:
# create poverty dataframe by filtering the columns I need
# for this one I can just keep the id field which will be used to join later, and the percent below poverty level
poverty_level = poverty.filter(['id', 'Estimate!!Below poverty level!!Population for whom poverty status is determined', 'Estimate!!Percent below poverty level!!Population for whom poverty status is determined'])

# rename columns
poverty_level = poverty_level.rename(columns={'Geographic Area Name': 'name',
                                              'Estimate!!Below poverty level!!Population for whom poverty status is determined': 'totalBelowPoverty',
                                              'Estimate!!Percent below poverty level!!Population for whom poverty status is determined': 'percentBelowPoverty'})

# some rows had a '-' for a null value
# replace these with nans
poverty_level = poverty_level.replace(r'-', np.nan)

# cast column to float
poverty_level['percentBelowPoverty'] = poverty_level['percentBelowPoverty'].astype(float)

In [33]:
poverty_level.describe()

,totalBelowPoverty,percentBelowPoverty
count,3.219000e+03,3219.000000
mean,1.421001e+04,16.390276
std,4.893290e+04,8.226827
min,7.000000e+00,2.300000
25%,1.645000e+03,11.100000
50%,4.133000e+03,14.900000
75%,1.053850e+04,19.500000
max,1.589956e+06,64.200000


### Age and Sex, 2018 5-year estimates

In [34]:
# read population data in
pop = pd.read_csv('../data/census-tables/2018-county-population-data.csv', encoding='latin-1', header=1)
list(pop.columns)

['id',
 'Geographic Area Name',
 'Estimate!!Total!!Total population',
 'Margin of Error!!Total MOE!!Total population',
 'Estimate!!Percent!!Total population',
 'Margin of Error!!Percent MOE!!Total population',
 'Estimate!!Male!!Total population',
 'Margin of Error!!Male MOE!!Total population',
 'Estimate!!Percent Male!!Total population',
 'Margin of Error!!Percent Male MOE!!Total population',
 'Estimate!!Female!!Total population',
 'Margin of Error!!Female MOE!!Total population',
 'Estimate!!Percent Female!!Total population',
 'Margin of Error!!Percent Female MOE!!Total population',
 'Estimate!!Total!!Total population!!AGE!!Under 5 years',
 'Margin of Error!!Total MOE!!Total population!!AGE!!Under 5 years',
 'Estimate!!Percent!!Total population!!AGE!!Under 5 years',
 'Margin of Error!!Percent MOE!!Total population!!AGE!!Under 5 years',
 'Estimate!!Male!!Total population!!AGE!!Under 5 years',
 'Margin of Error!!Male MOE!!Total population!!AGE!!Under 5 years',
 'Estimate!!Percent Male!!T

I only want to keep the total population and total population under 5 years, as well as the id field to be used in merging later.

In [35]:
# create new population dataframe by filtering the columns I need
pop = pop.filter(['id', 'Estimate!!Total!!Total population', 'Estimate!!Total!!Total population!!AGE!!Under 5 years'], axis =1)

# rename columns
pop = pop.rename(columns={'Estimate!!Total!!Total population': 'totalPop',
                                         'Estimate!!Total!!Total population!!AGE!!Under 5 years': 'totalPopUnder5'})

pop.head()

,id,totalPop,totalPopUnder5
0,0500000US01001,55200,3263
1,0500000US01003,208107,11609
2,0500000US01005,25782,1390
3,0500000US01007,22527,1275
4,0500000US01009,57645,3485


In [36]:
pop.describe()

,totalPop,totalPopUnder5
count,3.220000e+03,3220.000000
mean,1.013323e+05,6209.468323
std,3.260964e+05,20962.140633
min,7.500000e+01,4.000000
25%,1.121425e+04,623.750000
50%,2.595050e+04,1499.000000
75%,6.655225e+04,3910.250000
max,1.009805e+07,624745.000000


### Place of birth by education attainment in the United States, 2018 5-yr estimates (population 25 years and over in the United States)

In [37]:
# read education data in
education = pd.read_csv('../data/census-tables/2018-county-education-data.csv', encoding='latin-1', header=1)
list(education.columns)

['id',
 'Geographic Area Name',
 'Estimate!!Total',
 'Margin of Error!!Total',
 'Estimate!!Total!!Less than high school graduate',
 'Margin of Error!!Total!!Less than high school graduate',
 'Estimate!!Total!!High school graduate (includes equivalency)',
 'Margin of Error!!Total!!High school graduate (includes equivalency)',
 "Estimate!!Total!!Some college or associate's degree",
 "Margin of Error!!Total!!Some college or associate's degree",
 "Estimate!!Total!!Bachelor's degree",
 "Margin of Error!!Total!!Bachelor's degree",
 'Estimate!!Total!!Graduate or professional degree',
 'Margin of Error!!Total!!Graduate or professional degree',
 'Estimate!!Total!!Born in state of residence',
 'Margin of Error!!Total!!Born in state of residence',
 'Estimate!!Total!!Born in state of residence!!Less than high school graduate',
 'Margin of Error!!Total!!Born in state of residence!!Less than high school graduate',
 'Estimate!!Total!!Born in state of residence!!High school graduate (includes equivale

I only need to keep the id field, and the total number with less than high school education.

In [38]:
# create new education dataframe by filtering the columns I need
education = education.filter(['id', 'Estimate!!Total!!Less than high school graduate'], axis =1)

# rename columns
education = education.rename(columns={'Estimate!!Total!!Less than high school graduate': 'highSchoolEd'})

education.head()

,id,highSchoolEd
0,0500000US28151,6523.0
1,0500000US28111,1536.0
2,0500000US28019,1134.0
3,0500000US28057,3375.0
4,0500000US28015,1324.0


### Limited english speaking households, 2018 5-year estimates

In [39]:
# read english data in
english = pd.read_csv('../data/census-tables/2018-county-english-data.csv', encoding='latin-1', header=1)
list(english.columns)

['id',
 'Geographic Area Name',
 'Estimate!!Total!!All households',
 'Margin of Error!!Total MOE!!All households',
 'Estimate!!Percent!!All households',
 'Margin of Error!!Percent MOE!!All households',
 'Estimate!!Limited English-speaking households!!All households',
 'Margin of Error!!Limited English-speaking households MOE!!All households',
 'Estimate!!Percent limited English-speaking households!!All households',
 'Margin of Error!!Percent limited English-speaking households MOE!!All households',
 'Estimate!!Total!!All households!!Households speaking --!!Spanish',
 'Margin of Error!!Total MOE!!All households!!Households speaking --!!Spanish',
 'Estimate!!Percent!!All households!!Households speaking --!!Spanish',
 'Margin of Error!!Percent MOE!!All households!!Households speaking --!!Spanish',
 'Estimate!!Limited English-speaking households!!All households!!Households speaking --!!Spanish',
 'Margin of Error!!Limited English-speaking households MOE!!All households!!Households speaking

I only need to keep the id field, the total number of limited english housesholds, and the percent of limited-english speaking households.

In [40]:
# create new english dataframe by filtering the columns I need
english = english.filter(['id', 'Estimate!!Limited English-speaking households!!All households', 'Estimate!!Percent limited English-speaking households!!All households'], axis =1)

# rename columns
english = english.rename(columns={'Estimate!!Limited English-speaking households!!All households': 'totalLimitedEnglish',
                                    'Estimate!!Percent limited English-speaking households!!All households': 'percentLimitedEnglish'})

english.head()

,id,totalLimitedEnglish,percentLimitedEnglish
0,0500000US01001,138,0.7
1,0500000US01003,1120,1.4
2,0500000US01005,135,1.5
3,0500000US01007,47,0.7
4,0500000US01009,301,1.5


### Merging the wrangled census datasets together

In [41]:
census_data1 = pd.merge(pop, poverty_level, on='id', how='left')
census_data1.head()

,id,totalPop,totalPopUnder5,totalBelowPoverty,percentBelowPoverty
0,0500000US01001,55200,3263,8422.0,15.4
1,0500000US01003,208107,11609,21653.0,10.6
2,0500000US01005,25782,1390,6597.0,28.9
3,0500000US01007,22527,1275,2863.0,14.0
4,0500000US01009,57645,3485,8220.0,14.4


In [42]:
census_data2 = pd.merge(census_data1, education, on='id', how='left')
census_data2.head()

,id,totalPop,totalPopUnder5,totalBelowPoverty,percentBelowPoverty,highSchoolEd
0,0500000US01001,55200,3263,8422.0,15.4,4204.0
1,0500000US01003,208107,11609,21653.0,10.6,14310.0
2,0500000US01005,25782,1390,6597.0,28.9,4901.0
3,0500000US01007,22527,1275,2863.0,14.0,2650.0
4,0500000US01009,57645,3485,8220.0,14.4,7861.0


In [43]:
census_data3 = pd.merge(census_data2, english, on='id', how='left')
census_data3.head()

,id,totalPop,totalPopUnder5,totalBelowPoverty,percentBelowPoverty,highSchoolEd,totalLimitedEnglish,percentLimitedEnglish
0,0500000US01001,55200,3263,8422.0,15.4,4204.0,138,0.7
1,0500000US01003,208107,11609,21653.0,10.6,14310.0,1120,1.4
2,0500000US01005,25782,1390,6597.0,28.9,4901.0,135,1.5
3,0500000US01007,22527,1275,2863.0,14.0,2650.0,47,0.7
4,0500000US01009,57645,3485,8220.0,14.4,7861.0,301,1.5


In [44]:
census_data = pd.merge(census_data3, insurance_coverage, on='id', how='left')
census_data.head()

,id,totalPop,totalPopUnder5,totalBelowPoverty,percentBelowPoverty,highSchoolEd,totalLimitedEnglish,percentLimitedEnglish,name,totalUninsured
0,0500000US01001,55200,3263,8422.0,15.4,4204.0,138,0.7,"Autauga County, Alabama",3875
1,0500000US01003,208107,11609,21653.0,10.6,14310.0,1120,1.4,"Baldwin County, Alabama",20864
2,0500000US01005,25782,1390,6597.0,28.9,4901.0,135,1.5,"Barbour County, Alabama",2558
3,0500000US01007,22527,1275,2863.0,14.0,2650.0,47,0.7,"Bibb County, Alabama",1619
4,0500000US01009,57645,3485,8220.0,14.4,7861.0,301,1.5,"Blount County, Alabama",6303


In [45]:
# split name column into separate columns for county, and state
census_data['county'], census_data['state'] = census_data['name'].str.split(', ', 1).str

# grab last eleven digits from the id field to create a GEOID field for joining to shapefile
census_data['GEOID'] = census_data['id'].str[-5:]

census_data.head()

,id,totalPop,totalPopUnder5,totalBelowPoverty,percentBelowPoverty,highSchoolEd,totalLimitedEnglish,percentLimitedEnglish,name,totalUninsured,county,state,GEOID
0,0500000US01001,55200,3263,8422.0,15.4,4204.0,138,0.7,"Autauga County, Alabama",3875,Autauga County,Alabama,01001
1,0500000US01003,208107,11609,21653.0,10.6,14310.0,1120,1.4,"Baldwin County, Alabama",20864,Baldwin County,Alabama,01003
2,0500000US01005,25782,1390,6597.0,28.9,4901.0,135,1.5,"Barbour County, Alabama",2558,Barbour County,Alabama,01005
3,0500000US01007,22527,1275,2863.0,14.0,2650.0,47,0.7,"Bibb County, Alabama",1619,Bibb County,Alabama,01007
4,0500000US01009,57645,3485,8220.0,14.4,7861.0,301,1.5,"Blount County, Alabama",6303,Blount County,Alabama,01009


In [48]:
# calculate percent uninsured
census_data['percentUninsured'] = census_data['totalUninsured']/census_data['totalPop']*100

# calculate percent under 5
census_data['percentUnder5'] = census_data['totalPopUnder5']/census_data['totalPop']*100

# calculate percent with less than high school education
census_data['percentHighSchool'] = census_data['highSchoolEd']/census_data['totalPop']*100

census_data.head()

,id,totalPop,totalPopUnder5,totalBelowPoverty,percentBelowPoverty,highSchoolEd,totalLimitedEnglish,percentLimitedEnglish,name,totalUninsured,county,state,GEOID,percentUninsured,percentUnder5,percentHighSchool
0,0500000US01001,55200,3263,8422.0,15.4,4204.0,138,0.7,"Autauga County, Alabama",3875,Autauga County,Alabama,01001,7.019928,5.911232,7.615942
1,0500000US01003,208107,11609,21653.0,10.6,14310.0,1120,1.4,"Baldwin County, Alabama",20864,Baldwin County,Alabama,01003,10.025612,5.578380,6.876270
2,0500000US01005,25782,1390,6597.0,28.9,4901.0,135,1.5,"Barbour County, Alabama",2558,Barbour County,Alabama,01005,9.921651,5.391358,19.009386
3,0500000US01007,22527,1275,2863.0,14.0,2650.0,47,0.7,"Bibb County, Alabama",1619,Bibb County,Alabama,01007,7.186931,5.659875,11.763661
4,0500000US01009,57645,3485,8220.0,14.4,7861.0,301,1.5,"Blount County, Alabama",6303,Blount County,Alabama,01009,10.934166,6.045624,13.636916


### Join census data to county shapefile

In [49]:
# read shapefile in
counties = gpd.read_file('../data/county-shapefile/tl_2019_us_county.shp')
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016819475,29089486,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169270569,22849484,+40.7835474,-096.6886584,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [50]:
# filter to drop unneccessary columns
counties = counties.filter(['GEOID', 'NAME', 'NAMELSAD', 'ALAND', 'geometry'], axis=1)
counties.head()

,GEOID,NAME,NAMELSAD,ALAND,geometry
0,31039,Cuming,Cuming County,1477652222,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53069,Wahkiakum,Wahkiakum County,680962890,"POLYGON ((-123.43639 46.23820, -123.44759 46.2..."
2,35011,De Baca,De Baca County,6016819475,"POLYGON ((-104.56739 33.99757, -104.56772 33.9..."
3,31109,Lancaster,Lancaster County,2169270569,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31129,Nuckolls,Nuckolls County,1489645188,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [51]:
# join census data to shapefile
counties_joined = pd.merge(counties, census_data, on='GEOID', how='left')
counties_joined.head()

,GEOID,NAME,NAMELSAD,ALAND,geometry,id,totalPop,totalPopUnder5,totalBelowPoverty,percentBelowPoverty,highSchoolEd,totalLimitedEnglish,percentLimitedEnglish,name,totalUninsured,county,state,percentUninsured,percentUnder5,percentHighSchool
0,31039,Cuming,Cuming County,1477652222,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",0500000US31039,8991.0,533.0,615.0,7.0,771.0,79.0,2.1,"Cuming County, Nebraska",672.0,Cuming County,Nebraska,7.474141,5.928150,8.575242
1,53069,Wahkiakum,Wahkiakum County,680962890,"POLYGON ((-123.43639 46.23820, -123.44759 46.2...",0500000US53069,4189.0,140.0,323.0,7.8,282.0,11.0,0.6,"Wahkiakum County, Washington",297.0,Wahkiakum County,Washington,7.089998,3.342086,6.731917
2,35011,De Baca,De Baca County,6016819475,"POLYGON ((-104.56739 33.99757, -104.56772 33.9...",0500000US35011,2060.0,105.0,340.0,16.8,182.0,13.0,1.8,"De Baca County, New Mexico",161.0,De Baca County,New Mexico,7.815534,5.097087,8.834951
3,31109,Lancaster,Lancaster County,2169270569,"POLYGON ((-96.91075 40.78494, -96.91075 40.790...",0500000US31109,310094.0,20238.0,38952.0,13.1,12495.0,3370.0,2.7,"Lancaster County, Nebraska",22423.0,Lancaster County,Nebraska,7.231033,6.526408,4.029423
4,31129,Nuckolls,Nuckolls County,1489645188,"POLYGON ((-98.27367 40.08940, -98.27367 40.089...",0500000US31129,4275.0,226.0,509.0,12.2,235.0,10.0,0.5,"Nuckolls County, Nebraska",291.0,Nuckolls County,Nebraska,6.807018,5.286550,5.497076


In [53]:
# drop redundant columns
counties_joined = counties_joined.drop(['NAMELSAD', 'id', 'NAME', 'name'], axis=1)

# create a column with land area in square miles, converted from square meters
counties_joined['ALANDsquareMiles'] = counties_joined['ALAND'] / 2589988.110336

# create population density column
counties_joined['popDensity'] = counties_joined['totalPop']/counties_joined['ALANDsquareMiles']

counties_joined.head()

,GEOID,ALAND,geometry,totalPop,totalPopUnder5,totalBelowPoverty,percentBelowPoverty,highSchoolEd,totalLimitedEnglish,percentLimitedEnglish,totalUninsured,county,state,percentUninsured,percentUnder5,percentHighSchool,ALANDsquareMiles,popDensity
0,31039,1477652222,"POLYGON ((-97.01952 42.00410, -97.01952 42.004...",8991.0,533.0,615.0,7.0,771.0,79.0,2.1,672.0,Cuming County,Nebraska,7.474141,5.928150,8.575242,570.524712,15.759177
1,53069,680962890,"POLYGON ((-123.43639 46.23820, -123.44759 46.2...",4189.0,140.0,323.0,7.8,282.0,11.0,0.6,297.0,Wahkiakum County,Washington,7.089998,3.342086,6.731917,262.921242,15.932528
2,35011,6016819475,"POLYGON ((-104.56739 33.99757, -104.56772 33.9...",2060.0,105.0,340.0,16.8,182.0,13.0,1.8,161.0,De Baca County,New Mexico,7.815534,5.097087,8.834951,2323.106987,0.886743
3,31109,2169270569,"POLYGON ((-96.91075 40.78494, -96.91075 40.790...",310094.0,20238.0,38952.0,13.1,12495.0,3370.0,2.7,22423.0,Lancaster County,Nebraska,7.231033,6.526408,4.029423,837.560049,370.234947
4,31129,1489645188,"POLYGON ((-98.27367 40.08940, -98.27367 40.089...",4275.0,226.0,509.0,12.2,235.0,10.0,0.5,291.0,Nuckolls County,Nebraska,6.807018,5.286550,5.497076,575.155223,7.432776


In [55]:
# convert crs to WGS84 for web mapping
counties_joined = counties_joined.to_crs(epsg=4326)

# write joined census data to geojson
counties_joined.to_file("../data/census-outputs/counties.geojson", driver='GeoJSON')

Use command line to simplify file

`mapshaper counties.geojson -simplify dp 20% -o format=geojson counties-simplified.geojson`